In [ ]:
class ImdbRating:
    
    def __init__(self,title,year):
        self.title = title
        self.year = year
        self.unaccented = unidecode.unidecode(self.title).replace(" ","+")
        
    def getAndParse(self,url):
        result = requests.get(url)
        soup = BeautifulSoup(result.text, 'html.parser')
        return(soup)
    
    def GetRatingLink(self):  
        search_pattern = "https://www.imdb.com/search/title/?title={}&release_date={}-01-01,{}-12-31"
        search_link = search_pattern.format(self.unaccented,self.year,self.year)
        soup = self.getAndParse(search_link)
        title_number = soup.find("h3",class_="lister-item-header").a.get("href")
        rating_link = "https://www.imdb.com" + title_number + "ratings?ref_=tt_ov_rt"
        return(rating_link)
    
    def GetRating(self):
        soup = self.getAndParse(self.GetRatingLink())
        rating_distribution = [s.text for s in soup.find("div",class_="title-ratings-sub-page").table.find_all("div",class_="leftAligned")][1:]
        
        rating_table = [s.find("div",class_="bigcell").text for s in soup.find_all("td",class_="ratingTable")]
        
        ''' Male - Female '''
        rating_MFdifference = float(rating_table[5])-float(rating_table[10])
        ''' non-US - US '''
        rating_USdifference = float(rating_table[17])-float(rating_table[16])
        
        rating_labels = ["IMDB_" + str(i)+"star" for i in range(10,0,-1)] + ["IMDB_MF_diff","IMDB_US_diff"]
        rating_values = rating_distribution
        rating_values.extend([rating_MFdifference,rating_USdifference])
        
        output = pd.DataFrame([rating_values],columns=rating_labels)
        return(output) 